In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [3]:
model = keras.Sequential(
    [
        keras.Input(shape=(32, 32, 3)),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(10, activation="softmax"),
    ]
)

In [4]:
model.compile(
    optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"]
)

# Train the model
model.fit(x_train, y_train, batch_size=64, epochs=10, validation_split=0.1)

Epoch 1/10
704/704 [==============================] - 21s 16ms/step - loss: 0.2659 - accuracy: 0.3722 - val_loss: 0.2226 - val_accuracy: 0.5058
Epoch 2/10
704/704 [==============================] - 10s 14ms/step - loss: 0.2210 - accuracy: 0.5130 - val_loss: 0.2057 - val_accuracy: 0.5584
Epoch 3/10
704/704 [==============================] - 10s 14ms/step - loss: 0.2072 - accuracy: 0.5573 - val_loss: 0.1924 - val_accuracy: 0.6088
Epoch 4/10
704/704 [==============================] - 10s 14ms/step - loss: 0.1976 - accuracy: 0.5850 - val_loss: 0.1839 - val_accuracy: 0.6296
Epoch 5/10
704/704 [==============================] - 10s 14ms/step - loss: 0.1912 - accuracy: 0.6040 - val_loss: 0.1773 - val_accuracy: 0.6530
Epoch 6/10
704/704 [==============================] - 10s 14ms/step - loss: 0.1857 - accuracy: 0.6207 - val_loss: 0.1732 - val_accuracy: 0.6588
Epoch 7/10
704/704 [==============================] - 10s 14ms/step - loss: 0.1814 - accuracy: 0.6312 - val_loss: 0.1730 - val_accuracy:

In [5]:
import os
import numpy as np
import cv2

In [6]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
    return images

In [7]:
def preprocess_images(images):
    processed_images = []
    for img in images:
        resized_img = cv2.resize(img, (32, 32))
        processed_images.append(resized_img)
    return np.array(processed_images)

In [8]:
cd ..//data//raw//images//training

C:\Users\noahc\MonRepo\data\raw\images\training


In [9]:
#path_to_flip = "MonRepo//data//raw//images//training//flip"
#path_to_noflip = "MonRepo//data//raw//images//training//noflip"

In [10]:
# Load the flip images
flip_folder = 'flip'
flip_images = load_images_from_folder(flip_folder)
flip_images = preprocess_images(flip_images)
flip_labels = np.ones((len(flip_images),))

In [11]:
# Load the noflip images
noflip_folder = 'notflip'
noflip_images = load_images_from_folder(noflip_folder)
noflip_images = preprocess_images(noflip_images)
noflip_labels = np.zeros((len(noflip_images),))

In [12]:
# Concatenate the images and labels
x_train2 = np.concatenate((flip_images, noflip_images), axis=0)
y_train2 = np.concatenate((flip_labels, noflip_labels), axis=0)
y_train2 = keras.utils.to_categorical(y_train2, 2)

In [16]:
# Freeze the layers of the pre-trained model
for layer in model.layers[:-2]:
    layer.trainable = False
    
# Replace the final dense layer with a single output and a sigmoid activation function
model.layers[-1].units = 1
model.layers[-1].activation = 'sigmoid'
model.layers[-1]._name = 'output_layer'

In [17]:
# Compile the model for fine-tuning
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 2304)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2304)              0         
_________________________________________________________________
output_layer (Dense)         (None, 10)                2

In [20]:
# Fine-tune the model on the new data
model.fit(x_train2, y_train2, batch_size=64, epochs=10, validation_split=0.1)

Epoch 1/10


TypeError: in user code:

    C:\Users\noahc\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    C:\Users\noahc\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\noahc\anaconda3\envs\tf\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\noahc\anaconda3\envs\tf\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\noahc\anaconda3\envs\tf\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\noahc\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py:835 run_step  **
        